# Import some packages

In [ ]:
import numpy as np
import pandas as pd
import pandas as pd
import shutil
import time
from copy import deepcopy

device = 'cuda'
import torchvision.models as models

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split

# Load data

In [ ]:
train_df = pd.read_csv("../data/kaggle/train.csv")
test_df = pd.read_csv("../data/kaggle/test.csv")
print(f"Train dataframe has shape: {train_df.shape}")
print(f"Test dataframe has shape: {test_df.shape}")
display(train_df.head())
display(test_df.head())

# Transforming 
This will convert each protein sequence (a list of amino acids) to a matrix, where each row is the physical and chemical properties of the corresponding amino acid

In [ ]:
df_aa = pd.read_csv("../data/aminoacids.csv")
"""
0 = Name (Ignore)
1 = Abbr (Ignore)
2 = Letter
3 = Molecular Weight
4 = Molecular Formula
5 = Residue Formula (Ignore)
6 = Residue Weight (Ignore)
7 = pKa1
8 = pKb2
9 = pKx3
10 = pl4
11 = H
12 = VSC
13 = P1
14 = P2
15 = SASA
16 = NCISC
17 = carbon
18 = hydrogen
19 = nitrogen
20 = oxygen
21 = sulfur
"""
feature_list = [3, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
df_aa = df_aa.fillna(method="bfill")
display(df_aa)
df_aa.iloc[:, feature_list] = (df_aa.iloc[:, feature_list] - df_aa.iloc[:, feature_list].mean()) / df_aa.iloc[:, feature_list].std()
display(df_aa)

In [ ]:
# feature_list chooses which physical and chemical properties to inlcude
def getTransformDict(feature_list = [3, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]):
  transform_dict = {}
  for index, row in df_aa.iterrows():
    row_values = row.values
    letter = row_values[2]
    the_rest = [row_values[i] for i in feature_list]
    transform_dict.update({letter: np.array(the_rest, dtype="float")})
  transform_dict.update({None: np.zeros(shape=len(feature_list),dtype="float")})
  return transform_dict

In [ ]:
sequences_train = [list(string) for string in train_df["protein_sequence"].values.tolist()]
lst = []
transform_dict = getTransformDict()
for sequence in sequences_train:
    letterList = []
    for letter in sequence:
        letterList += transform_dict[letter].tolist()
    lst.append(letterList)
print(lst)

In [ ]:
# convert list of amino acids into a matrix of their corresponding physical and chemical properties
def convertSequences(sequences, MAX_LENGTH, transform_dict=None):
    if transform_dict is None:
        transform_dict = getTransformDict()
    sequenceList = []
    for sequence in sequences:
        letterMatrix = []
        for i in range(MAX_LENGTH):
            if i < len(sequence):
                letterMatrix += transform_dict[sequence[i]].tolist()
            else:
                letterMatrix += transform_dict[None].tolist()
        sequenceList.append(letterMatrix)
    print(np.array(sequenceList))
    return np.array(sequenceList)

In [ ]:
def getSequences(df, MAX_LENGTH=224):
  df.reset_index(inplace=True)
  sequences = [list(string) for string in df["protein_sequence"].values.tolist()]
  return convertSequences(sequences, MAX_LENGTH)

In [ ]:
def filterDataFrame(df, MIN_LENGTH=100, MAX_LENGTH=224):
    df["protein_sequence_len"] = df["protein_sequence"].apply(lambda x: len(x))
    df = df[df["protein_sequence_len"] <= MAX_LENGTH]
    df = df[df["protein_sequence_len"] >= MIN_LENGTH]
    if 'tm' in df:
        df['tm'] = (df['tm'] - df['tm'].mean())/df['tm'].std()
    return df

In [ ]:
from sklearn.model_selection import train_test_split

MAX_LENGTH = 224
MIN_LENGTH = 100
train_df = filterDataFrame(train_df, MIN_LENGTH=MIN_LENGTH, MAX_LENGTH=MAX_LENGTH)
X = getSequences(train_df, MAX_LENGTH=MAX_LENGTH)
y = train_df['tm'].values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
test_df = filterDataFrame(test_df, MIN_LENGTH=MIN_LENGTH, MAX_LENGTH=MAX_LENGTH)
X_test = getSequences(test_df)

## Loading and normalizing

In [ ]:
def toTensor(array):
    return torch.Tensor(np.array(array))

In [ ]:
import tensorflow as tf
from torch.utils.data import TensorDataset, DataLoader

print(torch.Tensor(np.array(X_train)))

trainset = TensorDataset(toTensor(X_train), torch.Tensor(y_train))
valset = TensorDataset(toTensor(X_val), torch.Tensor(y_val))
testset = TensorDataset(toTensor(X_test))

#TODO: revise batch_size
BATCH_SIZE = 32
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(valset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Using API Models

In [ ]:
def make_prediction(model, file):
    print(f'Saving results to {file}')
    submission = pd.DataFrame()
    submission["seq_id"] = test_df["seq_id"]
    submission["tm"] = model.predict(X_test)
    submission.to_csv(file, index=False)

In [ ]:
# from sklearn.linear_model import LinearRegression, SGDRegressor, LogisticRegression
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import make_pipeline
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import AdaBoostRegressor, ExtraTreesRegressor, GradientBoostingRegressor

In [ ]:
# make_prediction(DecisionTreeRegressor().fit(X_train, y_train), 'DTRpred.csv')
# make_prediction(RandomForestRegressor().fit(X_train, y_train), 'RFRpred.csv')
# make_prediction(AdaBoostRegressor().fit(X_train, y_train), 'ABRpred.csv')
# make_prediction(GradientBoostingRegressor().fit(X_train, y_train), 'GBRpred.csv')

# Define the model

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

    def forward(self, x):
        
        return x

In [ ]:
model = Net().to(device)

In [ ]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.7)

# Define the Training Functions

In [ ]:
%%time
best_val_loss = 100000
best_val_model = None
MAX_EPOCH = 30
loss_record = {'train': [], 'dev': []} # added
for epoch in range(MAX_EPOCH):  
    model.train()
    running_train_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()

        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = outputs.view(outputs.size(dim=0))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_train_loss += loss.item()
        out = outputs.detach()
        assert out.shape == labels.shape
        
    loss_record['train'].append(running_train_loss/len(trainset))
    
    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for inputs,labels in valloader:
            out = model(inputs.cuda()).cpu()
            out = out.detach().view(out.size(dim=0))
            running_val_loss += (torch.pow(out - labels, 2).sum().item()) ** 0.5
    loss_record['dev'].append(running_val_loss/len(valset))
    print("Epoch [{:>2} / {}]: Train loss = {:<25} Val Loss = {:<25}".format(epoch+1, MAX_EPOCH, running_train_loss/len(trainset), running_val_loss/len(valset)))
    if running_val_loss < best_val_loss:
        best_val_loss = running_val_loss
        best_val_model = deepcopy(model.state_dict())
    lr_scheduler.step()
    
print('Finished Training')

In [ ]:
# For plotting
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import math

In [ ]:
def plot_learning_curve(record, title=''):
    ''' Plot learning curve of your DNN (train & dev loss) '''
    x_1 = range(len(record['train']))
    x_2 = range(len(record['dev']))
    figure(figsize=(6, 4))
    plt.plot(x_1, record['train'], c='tab:red', label='train')
    plt.plot(x_2, record['dev'], c='tab:cyan', label='dev')
    
    # TODO: feel free to change this range to see the learning curve better
    y_min = min(min(record['train']), min(record['dev']))
    y_max = max(max(record['train']), max(record['dev']))
    plt.ylim(max(.95*y_min-0.01, 0), 1.05*y_max+0.01)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Learning curve of {}'.format(title))
    plt.legend()
    plt.show()

In [ ]:
plot_learning_curve(loss_record, title="CNN model")

In [ ]:
%%time
preds = []
model.load_state_dict(best_val_model)
model.eval()
with torch.no_grad():
    for inputs,labels in testloader:
        out = model(inputs.cuda()).cpu()
        out = torch.argmax(out,dim=1)
        preds.append(out.detach().cpu())
preds = torch.cat(preds, dim=0).numpy()

In [ ]:
def make_prediction(preds, file):
    print(f'Saving results to {file}')
    submission = pd.DataFrame()
    submission["tm"] = preds
    submission["seq_id"] = test_df["seq_id"]
    submission.to_csv(file, index=False)

make_prediction(preds, file="pred.csv")